<a href="https://colab.research.google.com/github/zahra-zarrabi/Bot_DeepLearning/blob/master/Gender_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/


In [ ]:
!kaggle datasets download -d ashishjangra27/gender-recognition-200k-images-celeba

In [ ]:
!unzip -qq gender-recognition-200k-images-celeba.zip

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


Hyper parameters

In [ ]:
batch_size = 32
width=height=224
epochs=10

Dataset

In [ ]:
data_generator = ImageDataGenerator(rescale=1./255,
                                    horizontal_flip=True)

train_data=data_generator.flow_from_directory('/content/Dataset/Train',
                                              batch_size=batch_size,
                                              shuffle=True,
                                              class_mode='categorical',
                                              target_size=(width,height))


test_data=data_generator.flow_from_directory('/content/Dataset/Test',
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=False,
                                              target_size=(width,height))


val_data=data_generator.flow_from_directory('/content/Dataset/Validation',
                                              batch_size=batch_size,
                                              shuffle=True,
                                              class_mode='categorical',
                                              target_size=(width,height))

Found 160000 images belonging to 2 classes.
Found 20001 images belonging to 2 classes.
Found 22598 images belonging to 2 classes.


In [ ]:
base_model =tf.keras.applications.ResNet50V2(
    include_top=True,
    weights="imagenet",
    input_shape=(width,height,3),
    pooling='evg',
)

102883328/102869336 [==============================] - 1s 0us/step


In [ ]:
# base_model.summary()

freeze

In [ ]:
for layer in base_model.layers[:-6]:
  layer.trainable=False

In [ ]:
model = tf.keras.Sequential([
                             base_model,
                             layers.Dense(2,activation='softmax')
])

In [ ]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
             optimizer=tf.keras.optimizers.Adam(),
             metrics=['accuracy']
             )

In [ ]:
model.fit(train_data,
          batch_size=batch_size,
          validation_data=val_data,
          epochs=5)

Epoch 1/5
5000/5000 [==============================] - 1149s 223ms/step - loss: 0.2186 - accuracy: 0.9374 - val_loss: 0.1508 - val_accuracy: 0.9463
Epoch 2/5
5000/5000 [==============================] - 1109s 222ms/step - loss: 0.1239 - accuracy: 0.9538 - val_loss: 0.1391 - val_accuracy: 0.9496
Epoch 3/5
5000/5000 [==============================] - 1107s 221ms/step - loss: 0.1076 - accuracy: 0.9596 - val_loss: 0.1292 - val_accuracy: 0.9526
Epoch 4/5
5000/5000 [==============================] - 1108s 222ms/step - loss: 0.0972 - accuracy: 0.9636 - val_loss: 0.1270 - val_accuracy: 0.9532
Epoch 5/5
5000/5000 [==============================] - 1110s 222ms/step - loss: 0.0873 - accuracy: 0.9671 - val_loss: 0.1306 - val_accuracy: 0.9542


In [ ]:
model.save('model_gender.h5')

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
y_pred=model.predict(test_data)
y_pred=np.argmax(y_pred,axis=1)
print(y_pred)
print('Confusion Matrix')
print(confusion_matrix(test_data.classes,y_pred))

target_names=list(test_data.class_indices.keys())
print('Classification Report')
print(classification_report(test_data.classes,y_pred,target_names=target_names))

[0 0 0 ... 1 1 1]
Confusion Matrix
[[11187   355]
 [  409  8050]]
Classification Report
              precision    recall  f1-score   support

      Female       0.96      0.97      0.97     11542
        Male       0.96      0.95      0.95      8459

    accuracy                           0.96     20001
   macro avg       0.96      0.96      0.96     20001
weighted avg       0.96      0.96      0.96     20001



In [ ]:
y_pred.shape